## Slow non-inactivting potassium in response to several voltage-clamps protocols
### Channel identity. ICG id: 	489, ModelDB id: 123623

In [ ]:
import matplotlib as mpl
import numpy as np
import pandas as pd
import time

from matplotlib import pyplot as plt

## Voltage-clamp protocol

In [ ]:
prot = ['v_act','v_inact','v_deact','v_ap','v_ramp']
prot_idx = 0
protocols = \
pd.read_csv('~/Dropbox/Master/mackelab/code/lfi-models/lfimodels/channelomics/protocols/k_channels/'+prot[prot_idx]+'.dat',sep='\t')

protocols = protocols.drop(protocols.columns[-1],axis=1)

# extract voltages and time
protocols1 = protocols.values

V = protocols1[:, 1:].T
t = protocols1[:, 0]   # attention: time step varies, but we will assume that it is constant
tstep = np.mean(np.diff(t)) # ms

## Parameters and kinetics

In [ ]:
# parameters
celsius = 37    # original temperature
temp = 37       # reference temperature
q10 = 2.3       # temperature sensitivity
tadj = q10**((celsius - temp)/10)

# length and diameter equal to 20 um
A_soma = np.pi*((20.*1e-4)**2)  # cm2

C = 1 # uF/cm2

g_L = 1e-1 # mS/cm2
E_L = -70 # not sure which one is used in Podlaski et al. 2017, mV

gbar_M = 0.004 # mS/cm2
E_K = -90. # mV
tau_max = 4000

noise_std = 0.001

###############################################################################
# kinetics
power = 1
param2 = 35.
param3 = 10.
param4 = 3.3
param5 = 20.

# gt
gt = np.array([1, 35, 10., 3.3, 20.])

def p_inf(x): 
    return 1. / (1 + np.exp(-(x + param2) / param3))

def tau_p(x): 
    arg = (x + param2) / param5
    return tau_max / (param4 * np.exp(arg) + np.exp(-arg))

## Voltage-clamp simulation

In [ ]:
t_sim = time.time()

p = np.zeros_like(V)

p[:, 0] = p_inf(V[:, 0])

len_p = len(p[:,0])

for i in range(1, t.shape[0]):
    # simple one step euler
#     pi = p[:, i-1]
#     p[:, i] = pi + tstep * tadj * (p_inf(V[:, i-1]) - pi 
#                                    + noise_std * np.random.randn(len_p) / (tstep**0.5)) / tau_p(V[:, i-1])
    pinf1 = p_inf(V[:, i-1]) + noise_std * np.random.randn(len_p) / (tstep**0.5)
    p[:, i] = pinf1 + (p[:, i-1] - pinf1) * np.exp(-tstep * tadj / tau_p(V[:, i-1]))

print(time.time() - t_sim)


fact_inward = 1 # inward current pre-multiplied by -1 (see page 15 of Podlasky et al. 2017)

I_K = fact_inward * tadj * gbar_M * p * (V - E_K)
I_K = I_K/np.max(I_K)
# I_L = tadj*g_L*(V-E_L)
# I_tot = (I_K + I_L)/np.max(I_K + I_L)

## Plot

In [ ]:
num_levels = len(V[:, 0])
cm1 = mpl.cm.viridis
col1 = [cm1(1.*i/num_levels) for i in range(num_levels)]

plt.figure(figsize=(18, 10))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
ax = plt.subplot(gs[0])
for i in range(num_levels):
    plt.plot(t, I_K[i,:], color = col1[i])
plt.ylabel('norm. current')
ax.set_xticks([])
# ax.set_yticks([0, 1.001*np.max(I_K)])

ax = plt.subplot(gs[1])
for i in range(num_levels):
    plt.plot(t, V[i, :],'r')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')

duration = np.max(t)
ax.set_xticks([0, duration/2, duration])
ax.set_yticks([0, np.max(V)])

plt.show()

## Checking model implementation against it (attention: discrepancy in temporal resolution)

In [ ]:
from lfimodels.channelomics.ChannelSingle import ChannelSingle
n_params = 8
cython = True
m = ChannelSingle(channel_type='kslow', n_params=n_params, cython=cython)

import numpy as np
t_sim = time.time()

out = m.gen_single(gt)
print(time.time() - t_sim)

import matplotlib.pyplot as plt
%matplotlib inline

K2 = out[prot[prot_idx]]['data']
t2 = out[prot[prot_idx]]['time']

In [ ]:
num_levels = len(V[:,0])
cm1 = mpl.cm.viridis
col1 = [cm1(1.*i/num_levels) for i in range(num_levels)]

plt.figure(figsize=(18, 10))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
ax = plt.subplot(gs[0])
for i in range(num_levels):
    plt.plot(t2, K2[i,], color = col1[i])
plt.ylabel('norm. current')
ax.set_xticks([])
ax.set_yticks([0, 1.001*np.max(I_K)])

ax = plt.subplot(gs[1])
for i in range(num_levels):
    plt.plot(t, V[i,],'r')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')

duration = np.max(t)
ax.set_xticks([0, duration/2, duration])
ax.set_yticks([0, np.max(V)])

plt.show()